# Clean csv generator

In [1]:
import os
import pandas as pd

# Dossiers
input_dir = './data/scopus'
output_dir = './data/cleanScopus'


for filename in os.listdir(input_dir):
    if filename.endswith('.csv') and filename != "MEDI.csv":
        input_path = os.path.join(input_dir, filename)
        
        df = pd.read_csv(input_path)
        
        clean_df = df[(df["prism:aggregationType"] == "Journal") & 
                      (df["prism:issn"].notnull() | df["prism:eIssn"].notnull())]
        
        output_path = os.path.join(output_dir, f"clean_{filename}")
        
        clean_df.to_csv(output_path, index=False)
        
print("Traitement terminé. Les fichiers nettoyés sont disponibles dans le dossier ./data/cleanScopus.")



KeyboardInterrupt: 